In [ ]:
import pandas as pd
import numpy as np


In [ ]:
pd.read_excel('RosterDate_2020.xlsx')

In [245]:
PlayerData = pd.read_excel(r"PlayerData_2019.xlsx")
PlayerData.head()

,Unnamed: 0,No.,Player,Age,Pos,G,GS,QBrec,Cmp,Att,...,Receiving_Y/G,Receiving_Ctch%,Receiving_Y/Tgt,Touch,Y/Tch,YScm,RRTD,Fmb,TeamName,TeamCode
0,0,12.0,Tom Brady,42,QB,16,16,12-4-0,373.0,613.0,...,NaN,NaN,NaN,26.0,1.3,34.0,3.0,4.0,New England Patriots,nwe
1,1,4.0,Jarrett Stidham,23,QB,3,0,NaN,2.0,4.0,...,NaN,NaN,NaN,2.0,-1.0,-2.0,0.0,0.0,New England Patriots,nwe
2,2,11.0,Julian Edelman,33,WR,16,13,NaN,2.0,2.0,...,69.8,65.4%,7.3,108.0,10.6,1144.0,6.0,3.0,New England Patriots,nwe
3,3,28.0,James White,27,RB,15,1,NaN,1.0,1.0,...,43.0,75.8%,6.8,139.0,6.5,908.0,6.0,1.0,New England Patriots,nwe
4,6,26.0,Sony Michel,24,RB,16,14,NaN,NaN,NaN,...,5.9,60.0%,4.7,259.0,3.9,1006.0,7.0,2.0,New England Patriots,nwe


In [246]:
PlayerData.columns

Index(['Unnamed: 0', 'No.', 'Player', 'Age', 'Pos', 'G', 'GS', 'QBrec', 'Cmp',
       'Att', 'Cmp%', 'Yds', 'TD', 'TD%', 'Int', 'Int%', 'Lng', 'Y/A', 'AY/A',
       'Y/C', 'Y/G', 'Rate', 'QBR', 'Sk', 'Yds.1', 'NY/A', 'ANY/A', 'Sk%',
       '4QC', 'GWD', 'Rushing_Att', 'Rushing_Yds', 'Rushing_TD', 'Rushing_Lng',
       'Rushing_Y/A', 'Rushing_Y/G', 'Rushing_A/G', 'Receiving_Tgt',
       'Receiving_Rec', 'Receiving_Yds', 'Receiving_Y/R', 'Receiving_TD',
       'Receiving_Lng', 'Receiving_R/G', 'Receiving_Y/G', 'Receiving_Ctch%',
       'Receiving_Y/Tgt', 'Touch', 'Y/Tch', 'YScm', 'RRTD', 'Fmb', 'TeamName',
       'TeamCode'],
      dtype='object')

In [247]:
Roster = pd.read_excel(r'RosterData_2020.xlsx')
Roster.head()

,Unnamed: 0,No.,Player,Age,Pos,G,GS,Wt,Ht,College/Univ,BirthDate,Yrs,AV,Drafted (tm/rnd/yr),Salary,TeamName,TeamCode
0,0,66,James Ferentz,31,C,NaN,NaN,300,6-2,Iowa,6/5/1989,6,NaN,NaN,NaN,New England Patriots,nwe
1,1,7,Jake Bailey,23,P,NaN,NaN,205,6-2,Stanford,6/18/1997,1,NaN,New England Patriots / 5th / 163rd pick / 2019,NaN,New England Patriots,nwe
2,2,NaN,Josh Uche,22,LB,NaN,NaN,245,6-1,Michigan,9/18/1998,Rook,NaN,New England Patriots / 2nd / 60th pick / 2020,NaN,New England Patriots,nwe
3,3,NaN,Brandon Copeland,29,LB,NaN,NaN,263,6-3,Pennsylvania,7/2/1991,7,NaN,NaN,NaN,New England Patriots,nwe
4,4,26,Sony Michel,25,RB,NaN,NaN,215,5-11,Georgia,2/17/1995,2,NaN,New England Patriots / 1st / 31st pick / 2018,NaN,New England Patriots,nwe


In [248]:
df = (Roster
      [Roster.Pos.isin(['QB', 'RB', 'WR', 'TE'])]
      [['TeamName','Player','Pos']]
      .merge(PlayerData[['TeamName','Player','Pos', 'G']],
        how = 'left')
      .sort_values(['TeamName','Pos', 'G'], ascending = [1,1,0])
      .rename(columns = {'G':'Games_2019'})
)
df['bit'] = 1 
df['rank'] = df[['TeamName','Pos','Games_2019','bit']].groupby(['TeamName','Pos']).agg('cumsum')['bit']
df = df.drop('bit', axis = 1 )
condition = (
    np.where(df.Pos == 'QB', 1, 
          np.where(df.Pos == 'RB', 2, 
                   np.where(df.Pos == 'WR', 4, 
                           np.where(df.Pos == 'TE', 2,0)))))
df['Games_2020'] = np.where(df['rank'] <= condition,16,0)
df

,TeamName,Player,Pos,Games_2019,rank,Games_2020
699,Arizona Cardinals,Kyler Murray,QB,16.0,1,16
700,Arizona Cardinals,Brett Hundley,QB,3.0,2,0
710,Arizona Cardinals,Drew Anderson,QB,NaN,3,0
711,Arizona Cardinals,Chris Streveler,QB,NaN,4,0
698,Arizona Cardinals,Chase Edmonds,RB,13.0,1,16
...,...,...,...,...,...,...
437,Washington Redskins,Jordan Veasy,WR,NaN,7,0
441,Washington Redskins,J.D. McKissic,WR,NaN,8,0
447,Washington Redskins,Jester Weah,WR,NaN,9,0
449,Washington Redskins,Darvin Kidsy,WR,NaN,10,0


In [249]:
# Use the player's team from the 2020 Depth Chart Roster
# Compute the opportunity share for each position: How many rushing attempts or targets are there, and split them across players. 
    # When stating out, do not change the targets, rather the share. This way you explicitly have to change the Team Total to increase 
    # a player's projection. If it doesn't make sense, it puts a ceiling on what you can change. 
PassingVolume = (
    PlayerData 
    [['Player', 'Pos', 'G', 'Att', 'Cmp', 'Yds', 'TD', 'Int']]
    .groupby(['Player', 'Pos'])
    .sum()
    .reset_index()

)
PassingVolume['PassAtt_G'] = PassingVolume.Att/PassingVolume.G
PassingVolume['CmpPct'] = PassingVolume.Cmp / PassingVolume.Att
PassingVolume['Yds_Cmp'] = PassingVolume.Yds / PassingVolume.Cmp
PassingVolume['TD_Cmp'] = PassingVolume.TD / PassingVolume.Cmp
PassingVolume['Int_Att'] = PassingVolume.Int / PassingVolume.Att
PassingVolume = PassingVolume.sort_values(['PassAtt_G', 'CmpPct', 'Yds_Cmp', 'TD_Cmp', 'Int_Att'], ascending = [0,0,0,0,1])


RushingVolume = (
    PlayerData
    [['Player', 'Pos', 'G', 'Rushing_Att', 'Rushing_Yds', 'Rushing_TD']]
    .groupby(['Player','Pos'])
    .sum()
    .reset_index()
)
RushingVolume['RushAtt_G'] = RushingVolume.Rushing_Att/RushingVolume.G
RushingVolume['Yds_Rush'] = RushingVolume.Rushing_Yds/RushingVolume.Rushing_Att
RushingVolume['TD_Rush'] = RushingVolume.Rushing_TD/RushingVolume.Rushing_Att
RushingVolume = RushingVolume.sort_values(['RushAtt_G', 'Yds_Rush', 'TD_Rush'], ascending = [0,0,0])


ReceivingVolume = (
    PlayerData
    [['Player', 'Pos', 'G', 'Receiving_Tgt', 'Receiving_Rec', 'Receiving_Yds', 'Receiving_TD']]
    .groupby(['Player','Pos'])
    .sum()
    .reset_index()
)
ReceivingVolume['Target_G'] = ReceivingVolume.Receiving_Tgt/ReceivingVolume.G
ReceivingVolume['RecPct'] = ReceivingVolume.Receiving_Rec/ReceivingVolume.Receiving_Tgt
ReceivingVolume['Yds_Rec'] = ReceivingVolume.Receiving_Yds/ReceivingVolume.Receiving_Rec
ReceivingVolume['TD_Rec'] = ReceivingVolume.Receiving_TD/ReceivingVolume.Receiving_Rec

ReceivingVolume = ReceivingVolume.sort_values(['Target_G', 'RecPct', 'Yds_Rec', 'TD_Rec'], ascending = [0,0,0,0])


In [250]:
ReceivingVolume.head(10)

,Player,Pos,G,Receiving_Tgt,Receiving_Rec,Receiving_Yds,Receiving_TD,Target_G,RecPct,Yds_Rec,TD_Rec
347,Michael Thomas,WR,16,185.0,149.0,1725.0,9.0,11.562500,0.805405,11.577181,0.060403
121,Davante Adams,WR,12,127.0,83.0,997.0,5.0,10.583333,0.653543,12.012048,0.060241
272,Julio Jones,WR,15,157.0,99.0,1394.0,6.0,10.466667,0.630573,14.080808,0.060606
130,DeAndre Hopkins,WR,15,150.0,104.0,1165.0,7.0,10.000000,0.693333,11.201923,0.067308
16,Allen Robinson,WR,16,154.0,98.0,1147.0,7.0,9.625000,0.636364,11.704082,0.071429
271,Julian Edelman,WR,16,153.0,100.0,1117.0,6.0,9.562500,0.653595,11.170000,0.060000
283,Keenan Allen,WR,16,149.0,104.0,1199.0,6.0,9.312500,0.697987,11.528846,0.057692
404,Robert Woods,WR,15,139.0,90.0,1134.0,2.0,9.266667,0.647482,12.600000,0.022222
466,Tyler Boyd,WR,16,148.0,90.0,1046.0,5.0,9.250000,0.608108,11.622222,0.055556
350,Mike Evans,WR,13,118.0,67.0,1157.0,8.0,9.076923,0.567797,17.268657,0.119403


In [251]:
df = df.merge(PassingVolume[['Player', 'Pos', 'PassAtt_G', 'CmpPct', 'Yds_Cmp', 'TD_Cmp', 'Int_Att']], on = ['Player', 'Pos'], how = 'left' )
df['PassAtt_2020'] = df.Games_2020 * df.PassAtt_G
df['PassCmp_2020'] = df.PassAtt_2020 * df.CmpPct 
df['PassYds_2020'] = df.PassCmp_2020 * df.Yds_Cmp
df['PassTD_2020'] = df.PassCmp_2020 * df.TD_Cmp
df['PassInt_2020'] = df.PassAtt_2020 * df.Int_Att
df = df.merge(RushingVolume[['Player', 'Pos', 'RushAtt_G', 'Yds_Rush', 'TD_Rush']], on = ['Player', 'Pos'], how = 'left' )
df['RushAtt_2020'] = df.Games_2020 * df.RushAtt_G
df['RushYds_2020'] = df.Yds_Rush * df.RushAtt_2020
df['RushTD_2020'] = df.TD_Rush * df.RushAtt_2020

df = df.merge(ReceivingVolume[['Player', 'Pos', 'Target_G', 'RecPct', 'Yds_Rec', 'TD_Rec']], on = ['Player', 'Pos'], how = 'left' )
df['Targets_2020'] = df.Games_2020 * df.Target_G
df['Rec_2020'] = df.RecPct * df.Targets_2020
df['RecYds_2020'] = df.Yds_Rec * df.Rec_2020
df['RecTD_2020'] = df.TD_Rec * df.Rec_2020
PlayerVolume = df
PlayerVolume.head()

,TeamName,Player,Pos,Games_2019,rank,Games_2020,PassAtt_G,CmpPct,Yds_Cmp,TD_Cmp,...,RushYds_2020,RushTD_2020,Target_G,RecPct,Yds_Rec,TD_Rec,Targets_2020,Rec_2020,RecYds_2020,RecTD_2020
0,Arizona Cardinals,Kyler Murray,QB,16.0,1,16,33.875000,0.643911,10.664756,0.057307,...,544.000000,4.000000,0.000000,NaN,NaN,NaN,0.000000,NaN,NaN,NaN
1,Arizona Cardinals,Brett Hundley,QB,3.0,2,0,3.666667,0.454545,9.800000,0.000000,...,0.000000,0.000000,0.000000,NaN,NaN,NaN,0.000000,NaN,NaN,NaN
2,Arizona Cardinals,Drew Anderson,QB,NaN,3,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Arizona Cardinals,Chris Streveler,QB,NaN,4,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Arizona Cardinals,Chase Edmonds,RB,13.0,1,16,0.000000,NaN,NaN,NaN,...,372.923077,4.923077,1.615385,0.571429,8.75,0.083333,25.846154,14.769231,129.230769,1.230769


In [253]:
TeamData = pd.read_excel('TeamData_2019.xlsx')

In [254]:
TeamData.columns

Index(['Unnamed: 0', 'TeamName', 'PF', 'Yds', 'Ply', 'Y/P', 'TO', 'FL', '1stD',
       'Passing_Cmp', 'Passing_Att', 'Passing_Yds', 'Passing_TD',
       'Passing_Int', 'Passing_NY/A', 'Passing_1stD', 'Rushing_Att',
       'Rushing_Yds', 'Rushing_TD', 'Rushing_Y/A', 'Rushing_1stD', 'Pen',
       'Yds.1', '1stPy', '#Dr', 'Sc%', 'TO%', 'Start', 'Time', 'Plays',
       'Yds.2', 'Pts', 'TeamCode'],
      dtype='object')

In [262]:
df = TeamData[['TeamName', 'Passing_Att', 'Rushing_Att']]
df.G = 16
df['PassAtt_G'] = df.Passing_Att/df.G
df['RushAtt_G'] = df.Rushing_Att/df.G
df['PlaysPerGame'] = (df.PassAtt_G + df.RushAtt_G)
df['PassRunSplit'] = df.PassAtt_G / df.PlaysPerGame
TeamData = df 
TeamData.sort_values('PassRunSplit')

,TeamName,Passing_Att,Rushing_Att,PassAtt_G,RushAtt_G,PlaysPerGame,PassRunSplit
4,Baltimore Ravens,440,596,27.5000,37.2500,64.7500,0.424710
28,San Francisco 49ers,478,498,29.8750,31.1250,61.0000,0.489754
21,Minnesota Vikings,466,476,29.1250,29.7500,58.8750,0.494692
9,Tennessee Titans,448,445,28.0000,27.8125,55.8125,0.501680
29,Seattle Seahawks,517,481,32.3125,30.0625,62.3750,0.518036
10,Indianapolis Colts,513,471,32.0625,29.4375,61.5000,0.521341
1,Buffalo Bills,513,465,32.0625,29.0625,61.1250,0.524540
14,Oakland Raiders,523,437,32.6875,27.3125,60.0000,0.544792
8,Houston Texans,534,434,33.3750,27.1250,60.5000,0.551653
13,Denver Broncos,504,409,31.5000,25.5625,57.0625,0.552026


In [256]:
df.columns

Index(['TeamName', 'Player', 'Pos', 'Games_2019', 'rank', 'Games_2020',
       'PassAtt_G', 'CmpPct', 'Yds_Cmp', 'TD_Cmp', 'Int_Att', 'PassAtt_2020',
       'PassCmp_2020', 'PassYds_2020', 'PassTD_2020', 'PassInt_2020',
       'RushAtt_G', 'Yds_Rush', 'TD_Rush', 'RushAtt_2020', 'RushYds_2020',
       'RushTD_2020', 'Target_G', 'RecPct', 'Yds_Rec', 'TD_Rec',
       'Targets_2020', 'Rec_2020', 'RecYds_2020', 'RecTD_2020'],
      dtype='object')

In [271]:
PlayerVolume.columns

Index(['TeamName', 'Player', 'Pos', 'Games_2019', 'rank', 'Games_2020',
       'PassAtt_G', 'CmpPct', 'Yds_Cmp', 'TD_Cmp', 'Int_Att', 'PassAtt_2020',
       'PassCmp_2020', 'PassYds_2020', 'PassTD_2020', 'PassInt_2020',
       'RushAtt_G', 'Yds_Rush', 'TD_Rush', 'RushAtt_2020', 'RushYds_2020',
       'RushTD_2020', 'Target_G', 'RecPct', 'Yds_Rec', 'TD_Rec',
       'Targets_2020', 'Rec_2020', 'RecYds_2020', 'RecTD_2020'],
      dtype='object')

In [281]:
df = (PlayerVolume[['TeamName', 'Player', 'Pos', 'Target_G', 'RushAtt_G']][PlayerVolume.Pos.isin(['RB','WR','TE'])]
 .merge(TeamData[['TeamName', 'PassAtt_G', 'RushAtt_G']]
        .rename(columns = {'RushAtt_G':'T_RushAtt_G'}), 
        how = 'left')
)
df['TargetShare'] = df.Target_G / df.PassAtt_G
df['RushingShare'] = df.RushAtt_G / df.T_RushAtt_G
df.sort_values(['TeamName', 'TargetShare'], ascending = [1,0]).head(20)

,TeamName,Player,Pos,Target_G,RushAtt_G,PassAtt_G,T_RushAtt_G,TargetShare,RushingShare
13,Arizona Cardinals,DeAndre Hopkins,WR,10.000000,0.133333,34.625,24.750,0.288809,0.005387
6,Arizona Cardinals,Larry Fitzgerald,WR,6.812500,0.000000,34.625,24.750,0.196751,0.000000
1,Arizona Cardinals,Kenyan Drake,RB,4.857143,12.142857,34.625,24.750,0.140278,0.490620
8,Arizona Cardinals,KeeSean Johnson,WR,4.200000,0.100000,34.625,24.750,0.121300,0.004040
9,Arizona Cardinals,Dan Arnold,WR,3.333333,0.000000,34.625,24.750,0.096270,0.000000
0,Arizona Cardinals,Chase Edmonds,RB,1.615385,4.615385,34.625,24.750,0.046654,0.186480
3,Arizona Cardinals,Maxx Williams,TE,1.187500,0.000000,34.625,24.750,0.034296,0.000000
7,Arizona Cardinals,Andy Isabella,WR,0.866667,0.266667,34.625,24.750,0.025030,0.010774
5,Arizona Cardinals,Trent Sherfield,WR,0.812500,0.000000,34.625,24.750,0.023466,0.000000
2,Arizona Cardinals,D.J. Foster,RB,NaN,NaN,34.625,24.750,NaN,NaN
